# Visualize Your Inference Output with Gradio Interface

![Gradio example](./images/gradio_example.png)

To quickly evaluate how well the model performs, we provide a visualization tool using [Gradio](https://github.com/gradio-app/gradio), an open-source Python package that allows for the fast deployment of machine learning models with a GUI. Currently, we support the following tasks:

* Segmentation task with `GradioSegmentationStrategy`
* Detection task with `GradioDetectionStrategy`

## Prerequisite

Please follow the instructions in [segmentation example](./seg_example.ipynb).

## Implementation

Let's demonstrate Gradio Interface with [segmentation example](./seg_example.ipynb). To adopt Gradio Interface, we provide `GradioInferAPP` which integrates inference model inherited from `base_gradio_inference_model.py`. By reusing most of the code in [segmentation example](./seg_example.ipynb), we can initiate Gradio Interface seamlessly.

### Setup imports

In [3]:
import os
from typing import Any, List, Optional, Sequence, Tuple

import numpy as np
import SimpleITK as sitk
from lungmask import LMInferer

from flavor.serve.apps import GradioInferAPP
from flavor.serve.inference.inference_models import GradioInferenceModel
from flavor.serve.inference.strategies import GradioSegmentationStrategy

### Setup inference model for Gradio Interface

In [4]:
class SegmentationGradioInferenceModel(GradioInferenceModel):
    def __init__(self):
        super().__init__()

    def define_inference_network(self):
        return LMInferer(modelname="LTRCLobes", fillmodel="R231")

    def set_categories(self):
        categories = [
            {"name": "Background", "display": False},
            {"name": "Left Upper Lobe", "display": True},
            {"name": "Left Lower Lobe", "display": True},
            {"name": "Right Upper Lobe", "display": True},
            {"name": "Right Middle Lobe", "display": True},
            {"name": "Right Lower Lobe", "display": True},
        ]
        return categories

    def set_regressions(self):
        return None

    def data_reader(
        self, files: Sequence[str], **kwargs
    ) -> Tuple[np.ndarray, List[str], Tuple[int]]:
        def sort_images_by_z_axis(filenames):

            sorted_reader_filename_pairs = []

            for f in filenames:
                dicom_reader = sitk.ImageFileReader()
                dicom_reader.SetFileName(f)
                dicom_reader.ReadImageInformation()

                sorted_reader_filename_pairs.append((dicom_reader, f))

            zs = [
                float(r.GetMetaData(key="0020|0032").split("\\")[-1])
                for r, _ in sorted_reader_filename_pairs
            ]

            sort_inds = np.argsort(zs)[::-1]
            sorted_reader_filename_pairs = [sorted_reader_filename_pairs[s] for s in sort_inds]

            return sorted_reader_filename_pairs

        pairs = sort_images_by_z_axis(files)

        readers, sorted_filenames = zip(*pairs)
        sorted_filenames = list(sorted_filenames)

        simages = [sitk.GetArrayFromImage(r.Execute()).squeeze() for r in readers]
        volume = np.stack(simages)
        volume = np.expand_dims(volume, axis=0)

        return volume, sorted_filenames, volume.shape[1:]

    def inference(self, x: np.ndarray) -> np.ndarray:
        return self.network.apply(np.squeeze(x, axis=0))

    def postprocess(self, out: Any, metadata: Optional[Any] = None) -> Any:
        # (1, h, w) -> (6, h, w)
        out = [
            np.expand_dims((out == i).astype(np.uint8), axis=0)
            for i in range(6)  # or len(self.categories)
        ]
        out = np.concatenate(out, axis=0)
        return out


### Initiate the service

In [7]:
app = GradioInferAPP(
    infer_function=SegmentationGradioInferenceModel(),
    output_strategy=GradioSegmentationStrategy,
)

app.run(port=int(os.getenv("PORT", 9111)))

OSError: Cannot find empty port in range: 9111-9111. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.

: 

## Interact with Gradio interface

You won't need additional JSON file for `GradioInferAPP`.  Go http://localhost:9111/. You can input any images you want and play around with the Gradio GUI running your own inference model. 